# Connexion_France_Travail_ADZUNA.ipynb

Ce script a pour objectif :
- d'extraire les offres d'emploi mises à disposition par :
  <br> _ l'API de **France Travail**
  <br>_ de l'API **ADZUNA**
- les stocker dans :
  <br> _ un fichier (**CSV**) en local
  <br> _ un fichier (**Parquet**) en local
  <br> _ dans une **BDD PostgreSQL** en local.

**/!\ Ajouts !** : 
- Voir ci-dessous.

1.	Ajout d’une table dédiée à la récupération d’offres d’emploi via API
- Nom de la nouvelle table : ‘web_scrapping_table’
- Modification du schéma général de la table avec :
  <br>_Ajout d’un champ indiquant l’origine des données : ‘origine_annonce’, ‘candidature_effectuee’
  <br>_Suppression des champs ‘date_candidature_jour’, ‘date_candidature_mois’ et ‘date_candidature_annee’
- Modification de la fonction init_db(engine, table_name) : OK !!!

2.	Insérer les offres via api dans la table dédiée aux offres API
- Ne pas remplir les colonnes ‘manuelles’ pour le suivi.
- Remplir le champ ‘origine_annonce’ avec la valeur ‘API’
- Calculer les embeddings
- Création de la fonction ‘save_to_postgres_upsert_initial_api’ : OK !!

3.	Créer une fonction générique pour calculer le score de similarité. Cette fonction peut être appelée sur une table quelconque
- Une fonction “compute_similarity(reference_text, engine, table_name)” calcule dans un premier temps la similitude entre texte de référence et description de chaque offre.
- Une fonction « update_similarity » intègre ce score de similitude dans la table dédiée de la base de données.
- Création OK !!

4.	Créer une fonction générique d’export d’une table de la base de données
- Création OK !!

5.	Créer une fonction de mise à jour de la table dédiée API à partir du fichier de suivi
- Création OK !!

Comment ?
1. Sur la base de critères spécifiques (mots clés, localisation, etc...), 
    - lancement d'une requête pour obtenir les offres d'emploi correspondantes via l'API France Travail
    - lancement d'une requête pour obtenir les offres d'emploi correspondantes via l'API Adzuna
2. Une fois les offres trouvées, vérification et suppression des doublons.
3. Une sauvegarde en local des offres sont stockées dans un fichier (**CSV**).
4. Une sauvegarde en local des offres sont stockées dans un fichier (**Parquet**).
5. Une sauvegarde dans une base de données **PostgreSQL** est également effectuée en local.

Les URL (FRANCE TRAVAIL) utiles sont :
- https://francetravail.io/data/api/offres-emploi
- https://francetravail.io/data/api/offres-emploi/documentation#/

Les URL (ADZUNA) utiles sont :
- https://developer.adzuna.com/overview
- https://developer.adzuna.com/docs/search
- https://developer.adzuna.com/activedocs#!/adzuna/search
- https://developer.adzuna.com/overview
- https://www.adzuna.fr/details/5376850320?utm_medium=api&utm_source=6d1ef246

URL utile pour récupérer les informations géographiques:
- https://www.data.gouv.fr/datasets/contours-communes-france-administrative-format-admin-express-avec-arrondissements/

## Imports

In [1277]:
import os
import requests
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine, Table, MetaData, text
from sqlalchemy.dialects.postgresql import insert as pg_insert
from sqlalchemy.exc import SQLAlchemyError
import spacy
from sentence_transformers import SentenceTransformer, util
import logging
import time
import numpy as np
import http.client
import json
import hashlib
import geopandas as gpd
from shapely.geometry import Point

## Procédure

### Configuration

In [1278]:
##################  VARIABLES  ##################
# France Travail
FT_CLIENT_ID = os.environ.get("FT_CLIENT_ID")
FT_CLIENT_SECRET = os.environ.get("FT_CLIENT_SECRET")
FT_SCOPE = os.environ.get("FT_SCOPE")

# Adzuna
ADZUNA_CLIENT_ID = os.environ.get("ADZUNA_CLIENT_ID")
ADZUNA_CLIENT_SECRET = os.environ.get("ADZUNA_CLIENT_SECRET")

# Param Database PostgreSQL
DB_NAME = os.environ.get("DB_NAME", "jobsdb")
DB_USER = os.environ.get("DB_USER","jobsuser")
DB_PASS = os.environ.get("DB_PASS", "jobspass")
DB_HOST = os.environ.get("DB_HOST", "localhost")
DB_PORT = os.environ.get("DB_PORT","5432")

# Nom table
DB_TABLE_NAME = "offres"

### Configuration Logging

In [1279]:
# Logging
LOG_DIR = "logs"
os.makedirs(LOG_DIR, exist_ok=True)
logging.basicConfig(
    filename=os.path.join(LOG_DIR, f"pipeline_{datetime.now().strftime('%Y-%m-%d')}.log"),
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

### NLP ET embeddings

In [1280]:
# NLP & embeddings
nlp = spacy.load("fr_core_news_sm")
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

### Texte de référence (CV)

In [1281]:
# Offre de référence
reference_text = """
-	Data Analyst
-	Data Scientist
-	Data Analyst en reconversion
-	10 ans d’expérience industrie automobile & achats
-	Expert en dashboards et optimisation de performance

-	Localisation :
o	Ile-de-France
o	Yvelines
o	Poissy
o	78

-	Compétences :
o	Data Analysis & BI : Power BI (DAX, Power Query), Excel avancé, SQL.
o	Data Visualization : Création de tableaux de bord et KPIs pour la prise de décision.
o	Statistiques & Prévisions : Analyses quantitatives, modèles prédictifs, préventions des risques.
o	Conception d’outils d’aide à la décision et de tableaux de bord stratégiques
o	Méthodologies : Gestion de projets analytiques, reporting automatisé.
o	Exploitation de solutions de Data Science & Intelligence Artificielle : Machine Learning, modèles prédictifs, classification, régression, clustering
o	Analyses et modélisations statistiques avancées : Python, outils BI, Dataiku
o	Gestion et structuration de données massives : SQL Server, MySQL, Cloud Azure (Machine Learning Studio).
o	Développement et optimisation d’algorithmes : pour la performance et l’automatisation.
o	Conception d’outils d’aide à la décision et de tableaux de bord stratégiques pour orienter les choix business.

-	Informatique :
o	Langages de programmation : Python, Java, SQL
o	Logiciel : Power BI, Excel, Dataiku, Jupyter Notebook
o	Cloud : Azure, Google Cloud Platform

-	Diplômes et Formations :
o	Certification Microsoft Analyste de Données Power BI (PL300)
o	Bac+4 - Concepteur développeur en IA et analyse Big Data
o	Bac+5 –Master 2 Electronique Electrotechnique et Automatique

-	Atouts :
o	Autonome et rigoureux dans la gestion de projets.
o	Vulgarise des résultats complexes pour des non-spécialistes.
o	Organise et priorise les tâches orientées résultats.
o	En veille active sur l’IA et les nouvelles technologies.
o	Curieux
o	Autonome et rigoureux
o	Force de proposition
o	A l'écoute
o	Esprit d'équipe
-	Langues : Anglais courant
-	Expériences professionnelles : 
o	Acheteur de composants : 
	Analyser et structurer des données fournisseurs pour l’optimisation des coûts.
	Développer des tableaux de bord (KPI, suivi de performance) automatisés.
	Communiquer des insights aux équipes finance, qualité et production.
	Réaliser des économies supérieures de 20 % aux objectifs fixés.
	Analyser et structurer des données massives pour optimiser les coûts et la performance fournisseurs.
	Créer et automatiser de tableaux de bord (Power BI, Excel avancé) pour le suivi des KPIs.
	Développer des modèles prédictifs pour la prévision des coûts et l’analyse de tendances.
	Gérer des projets interfonctionnels (production, finance, qualité), générant +20 % d’économies au-delà des objectifs.
o	Responsable de développement de machines électrique :
	Analyser et valider des données issues des tests de performance produits.
	Automatiser des traitements statistiques pour réduire les erreurs de reporting.
	Mettre en place de modèles prédictifs pour améliorer la fiabilité des composants.
	Concevoir des solutions analytiques pour optimiser la durabilité et la performance des composants.
	Collaborer en mode Agile avec équipes R&D et Data pour intégrer l’analyse dans l’amélioration continue.
	Analyse statistique et validation de données issues de tests de performance.
-	Centres d’intérêt :
o	Théâtre : Improvisation
o	Moto : Sorties en groupe
"""
reference_text_clean = " ".join([token.lemma_ for token in nlp(reference_text.lower()) if not token.is_stop])

### Paramètres de recherche

In [1282]:
# ---------------------------
# PARAMETRES DE RECHERCHE
# ---------------------------
# Paramètres de recherche
# JOB_QUERY = "data analyst"
COMMUNE = "78300"
DISTANCE = 100000

# Nombre d'annonces par page requise
BLOC_PAGINATION = 50

# Nombre de pages max
MAX_PAGES = 20   # Limiter le nombre de pages récupérées

### Paramètres de sauvegarde

In [1283]:
# ---------------------------
# PARAMETRES DE SAUVEGARDE
# ---------------------------
# Répertoires
# Processed_data
IMPORT_TRACKING_DIR_PROC = "../data/processed_data/suivi_candidature/input_tracking_file"
EXPORT_TRACKING_DIR_PROC = "../data/processed_data/suivi_candidature/output_tracking_file"

### Authentification France Travail

In [1284]:
# ---------------------------
# AUTH FRANCE TRAVAIL
# ---------------------------
def get_ft_token(retries=3, wait=5):
    url = "https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=/partenaire"
    data = {
        "grant_type": "client_credentials",
        "client_id": FT_CLIENT_ID,
        "client_secret": FT_CLIENT_SECRET,
        "scope": FT_SCOPE,
    }
    for attempt in range(retries):
        try:
            r = requests.post(url, data=data)
            r.raise_for_status()
            return r.json()["access_token"]
        except requests.RequestException as e:
            logging.warning(f"Erreur OAuth attempt {attempt+1}: {e}")
            time.sleep(wait)
    raise RuntimeError("Impossible d'obtenir un token OAuth après plusieurs essais.")

### Lancement requête API France Travail

In [1285]:
# ---------------------------
# API CALL FRANCE TRAVAIL
# ---------------------------
def fetch_france_travail_jobs(query, token, max_pages=MAX_PAGES):
    headers = {"Authorization": f"Bearer {token}"}
    try:        
        all_jobs = []
        b_stop_criteria = False
        
        for page in range(1, max_pages + 1):
            if b_stop_criteria == False:    
                url = f"https://api.francetravail.io/partenaire/offresdemploi/v2/offres/search"
                params = {
                    "motsCles": query,
                    "commune": COMMUNE,
                    "distance" : DISTANCE,
                    "range": f"{(page-1)*BLOC_PAGINATION}-{page*BLOC_PAGINATION-1}"  # pagination par blocs de 50
                }
                r = requests.get(url, headers=headers, params=params)
                r.raise_for_status()
                data = r.json()
                offres = data.get("resultats", [])
                    
                for o in offres:
                    all_jobs.append({
                        "origine_annonce" : "API",
                        "source": "France Travail",
                        "recherche": f"{query}",
                        "id":o.get("id") if o.get("id") is not None else "None",    
                        "titre": o.get("intitule") if o.get("intitule") is not None else "None",                     
                        "description": o.get("description") if o.get("description") is not None else "None", 
                        "entreprise": o.get("entreprise", {}).get("nom") if o.get("entreprise", {}).get("nom") is not None else "None", 
                        "lieu": o.get("lieuTravail", {}).get("libelle") if o.get("lieuTravail", {}).get("libelle") is not None else "None", 
                        "latitude": o.get("lieuTravail", {}).get("latitude") if o.get("lieuTravail", {}) is not None else "None", 
                        "longitude": o.get("lieuTravail", {}).get("longitude") if o.get("lieuTravail", {}) is not None else "None", 
                        "type_contrat_libelle": o.get("typeContratLibelle") if o.get("typeContratLibelle") is not None else "None", 
                        "date_publication": o.get("dateCreation") if o.get("dateCreation") is not None else "None",    
                        "url": o.get("origineOffre").get("urlOrigine") if o.get("origineOffre") is not None else "None",
                        "secteur_activites": o.get("secteurActiviteLibelle") if o.get("dateCreation") is not None else "None"
                    })
    
                # Si le nombre d'offres est inférieur au nombre max d'offre par pages, c'est un signe qu'il n'y a plus d'offres à extraire après la page actuelle.
                if len(offres) < BLOC_PAGINATION:
                    b_stop_criteria = True
                
        return all_jobs
    except requests.RequestException as e:
        logging.error(f"Erreur API France Travail: {e}")
        return []

### Lancement de requête Adzuna

In [1286]:
# ---------------------------
# API CALL ADZUNA
# ---------------------------
def fetch_adzuna_jobs(query, max_pages=MAX_PAGES):
    headers = {"Accept": "application/json"}
    try:        
        all_jobs = []
        b_stop_criteria = False
        
        for page in range(1, max_pages + 1):
            if b_stop_criteria == False:    
                url = f"https://api.adzuna.com/v1/api/jobs/fr/search/{page}"
                params = {
                    "app_id" : ADZUNA_CLIENT_ID,
                    "app_key" : ADZUNA_CLIENT_SECRET,
                    "title_only": query,
                    "where": COMMUNE,
                    "results_per_page" : BLOC_PAGINATION,
                    "distance" : DISTANCE
                }
                r = requests.get(url,params=params)
                r.raise_for_status()
                data = r.json()
                offres = data.get("results")
    
                for o in offres:
                    all_jobs.append({
                        "origine_annonce" : "API",
                        "source": "Adzuna",
                        "recherche":f"{query}",
                        "id" : o.get("id") if o.get("id") is not None else "None", 
                        "titre" : o.get("title") if o.get("title") is not None else "None", 
                        "description" : o.get("description") if o.get("description") is not None else "None", 
                        "entreprise": o.get("company").get("display_name") if o.get("company") is not None else "None",
                        "lieu" : o.get("location").get("display_name") if o.get("location") is not None else "None",    
                        "latitude" : o.get("latitude") if o.get("latitude") is not None else "None", 
                        "longitude" : o.get("longitude") if o.get("longitude") is not None else "None",
                        "type_contrat_libelle" : o.get("contract_type") if o.get("contract_type") is not None else "None",                
                        "date_publication" : o.get("created") if o.get("created") is not None else "None",  
                        "url" : o.get("redirect_url") if o.get("redirect_url") is not None else "None",  
                        "secteur_activites" : o.get("category").get("label") if o.get("category") is not None else "None",
                    })
                    
                # Si le nombre d'offres est inférieur au nombre max d'offre par pages, c'est un signe qu'il n'y a plus d'offres à extraire après la page actuelle.
                if len(offres) < BLOC_PAGINATION:
                    b_stop_criteria = True
                
        return all_jobs
    except requests.RequestException as e:
        logging.error(f"Erreur API Adzuna: {e}")
        return []

### Déduplication

In [1287]:
# ---------------------------
# DÉDUPLICATION
# ---------------------------
def deduplicate(jobs):
    seen = set()
    deduped = []
    for job in jobs:
        key_str = f"{job['titre']}_{job['entreprise']}_{job['latitude']}_{job['longitude']}_{job['date_publication']}"
        key = hashlib.md5(key_str.encode()).hexdigest()
        if key not in seen:
            seen.add(key)
            deduped.append(job)
    return deduped

### Ajout infos localisation

In [1288]:
# ------------------------------------------------------------------------------
# RECHERCHE INFOS LOCALISATION SUPPLEMENTAIRES (commune, code_postal, departement)
# ------------------------------------------------------------------------------

def get_localization_info(df):    
    PATH_COMMUNES = "../data/raw_data/location_data/COMMUNE_FRMETDROM.shp"
    
    # Extract ccoordonées GPS
    coord = df[['longitude','latitude']]
    
    # Transformer en GeoDataFrame (EPSG:4326 = WGS84 = lat/lon)
    gdf_points = gpd.GeoDataFrame(
        coord,
        geometry=[Point(xy) for xy in zip(coord.longitude, coord.latitude)],
        crs="EPSG:4326"
    )

    # Charger le shapefile des communes
    communes = gpd.read_file(PATH_COMMUNES).to_crs(epsg=4326)

    # Jointure spatiale
    result = gpd.sjoin(gdf_points, communes, how="left", predicate="within")

    # Garder les colonnes utiles
    final = result[["NOM_M", 
                    "INSEE_COM", 
                    "INSEE_DEP"]].rename(columns = {"NOM_M":"commune",
                                                    "INSEE_COM":"code_postal",
                                                    "INSEE_DEP":"departement"})

    return pd.merge(df, final, left_index=True, right_index=True)

### Nettoyage de texte

In [1289]:
def clean_text(text):
    if not text:
        return ""
    doc = nlp(text.lower())
    return " ".join([token.lemma_ for token in doc if not token.is_stop])

### Calcul embeddings

In [1290]:
def compute_embedding(text):
    return model.encode([text], convert_to_numpy=True,show_progress_bar=False)[0].tolist()

### Initialisation database

In [1291]:
# --- Initialisation DB ---
# SUPPRESSION DE Date_creation TIMESTAMP
def init_db(engine, table_name):
    with engine.begin() as conn:
        
        # Activer l'extension PGVector
        conn.execute(text("CREATE EXTENSION IF NOT EXISTS vector;"))

        # Créer la table
        conn.execute(text(f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            id TEXT PRIMARY KEY,
            origine_annonce TEXT,
            source TEXT,
            recherche TEXT,
            titre TEXT,
            description TEXT,
            entreprise TEXT,
            lieu TEXT,
            latitude FLOAT(4),
            longitude FLOAT(4),
            commune TEXT,
            code_postal TEXT,
            departement TEXT,
            type_contrat_libelle TEXT,
            date_publication TIMESTAMP,
            url TEXT,
            secteur_activites TEXT,
            last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            embedding vector(384),
            similitude FLOAT,
            candidature_envisagee TEXT,
            type_contrat TEXT,
            experience_requise TEXT,
            candidature_effectuee TEXT,
            date_candidature DATE,
            nom_cv TEXT,
            nom_lm TEXT,
            nom_fichier_offre TEXT,
            date_relance_prevue DATE,
            date_relance_effectuee DATE,
            reponse_recue TEXT,
            date_reponse_entreprise DATE,
            etape_atteinte TEXT,
            nom_coord_recruteur TEXT,
            notes_perso TEXT,
            resultat_final TEXT,
            nb_jours_candidature_reponse INTEGER,
            nb_jours_candidature_resultat_final INTEGER,
            score_adequation_poste_profil TEXT,
            priorite_offre TEXT,
            mots_cles_poste TEXT,
            motivation TEXT
        )
        """))

### Sauvegarde en base PostgreSQL

In [1292]:
def save_to_postgres_upsert_initial_api(df, engine, table_name):
    """
    Sauvegarde un DataFrame pandas dans PostgreSQL avec UPSERT.
    Met à jour last_updated pour chaque ligne insérée ou modifiée.
    
    Args:
        df (pd.DataFrame): données à insérer
        engine (sqlalchemy.Engine): moteur SQLAlchemy connecté à PostgreSQL
        table_name (str): nom de la table cible
    
    Returns:
        int: nombre de lignes insérées ou mises à jour
    """
    
    if df.empty:
        logging.info("📭 DataFrame vide, rien à insérer.")
        return 0

    # Nettoyage des NaN
    df = df.where(pd.notnull(df), None)
    
    metadata = MetaData()
    table = Table(table_name, metadata, autoload_with=engine)
    now = datetime.utcnow()
    count = 0

    try:        
        with engine.begin() as conn:
            for row in df.to_dict(orient="records"):
                row["last_updated"] = now
                
                # Calcul embedding uniquement si nouvelle offre
                if not row.get("embedding"):
                    row["embedding"] = compute_embedding(row["description"])
                    
                stmt = pg_insert(table).values(row)
                stmt = stmt.on_conflict_do_update(
                    index_elements=['id'],
                    set_={
                        'origine_annonce': stmt.excluded.origine_annonce,
                        'source': stmt.excluded.source,
                        'recherche':stmt.excluded.recherche,
                        'titre': stmt.excluded.titre,
                        'description': stmt.excluded.description,
                        'entreprise': stmt.excluded.entreprise,
                        'lieu': stmt.excluded.lieu,
                        'latitude': stmt.excluded.latitude,
                        'longitude': stmt.excluded.longitude,   
                        'commune': stmt.excluded.commune,   
                        'code_postal': stmt.excluded.code_postal,   
                        'departement': stmt.excluded.departement,                 
                        'type_contrat_libelle': stmt.excluded.type_contrat_libelle,
                        'date_publication': stmt.excluded.date_publication,
                        'url': stmt.excluded.url,
                        'secteur_activites': stmt.excluded.secteur_activites,
                        # forcé à chaque exécution, même sans changement d'autres colonnes
                        'last_updated': now,
                        'embedding': stmt.excluded.embedding
                    }
                )
                conn.execute(stmt)
                count += 1
        logging.info(f"{count} offres insérées/mises à jour dans PostgreSQL.")
        return count

    except SQLAlchemyError as e:
        logging.error(f"❌ Erreur lors de l'UPSERT : {str(e)}")
        return 0

### Calcul similarité

In [1293]:
def update_similarity(df, engine, table_name):
    """
    Sauvegarde un DataFrame pandas dans PostgreSQL avec UPSERT.
    Met à jour last_updated pour chaque ligne insérée ou modifiée.
    
    Args:
        df (pd.DataFrame): données à insérer
        engine (sqlalchemy.Engine): moteur SQLAlchemy connecté à PostgreSQL
        table_name (str): nom de la table cible
    
    Returns:
        int: nombre de lignes insérées ou mises à jour
    """
    
    if df.empty:
        logging.info("📭 DataFrame vide, rien à insérer.")
        return 0

    # Nettoyage des NaN
    df = df.where(pd.notnull(df), None)
    
    metadata = MetaData()
    table = Table(table_name, metadata, autoload_with=engine)
    now = datetime.utcnow()
    count = 0
    
    print("Lancement MAJ du score de similarité dans la base !")
    
    try:        
        with engine.begin() as conn:
            for row in df.to_dict(orient="records"):
                row["last_updated"] = now
                
                # Calcul embedding uniquement si nouvelle offre
                if not row.get("embedding"):
                    row["embedding"] = compute_embedding(row["description"])
                    
                stmt = pg_insert(table).values(row)
                stmt = stmt.on_conflict_do_update(
                    index_elements=['id'],
                    set_={
                        'similitude': stmt.excluded.similitude, 
                        # forcé à chaque exécution, même sans changement d'autres colonnes
                        'last_updated': now,
                    }
                )
                conn.execute(stmt)
                count += 1
        logging.info(f"{count} offres insérées/mises à jour dans PostgreSQL.")
        return count

    except SQLAlchemyError as e:
        logging.error(f"❌ Erreur lors de l'UPSERT : {str(e)}")
        return 0

In [1294]:
def compute_similarity(reference_text, engine, table_name):
    ref_emb = compute_embedding(reference_text)
    ref_emb_str = "[" + ",".join(map(str, ref_emb)) + "]"  # convertir en string pour PGVector

    query = text(f"""
        SELECT     
                id, origine_annonce, source, recherche, titre, description, entreprise, 
                lieu, latitude, longitude, commune, code_postal, departement, 
                type_contrat_libelle, date_publication, url, secteur_activites, 
                last_updated, embedding, similitude, candidature_envisagee, type_contrat, 
                experience_requise, candidature_effectuee, date_candidature, 
                nom_cv, nom_lm, nom_fichier_offre, date_relance_prevue, 
                date_relance_effectuee, reponse_recue, date_reponse_entreprise, 
                etape_atteinte, nom_coord_recruteur, notes_perso, resultat_final, 
                nb_jours_candidature_reponse, nb_jours_candidature_resultat_final, 
                score_adequation_poste_profil, priorite_offre, mots_cles_poste, 
                motivation, simil_temp
        FROM (
                SELECT 
                    id, origine_annonce, source, recherche, titre, description, entreprise, 
                    lieu, latitude, longitude, commune, code_postal, departement, 
                    type_contrat_libelle, date_publication, url, secteur_activites, 
                    last_updated, embedding, similitude, candidature_envisagee, type_contrat, 
                    experience_requise, candidature_effectuee, date_candidature, 
                    nom_cv, nom_lm, nom_fichier_offre, date_relance_prevue, 
                    date_relance_effectuee, reponse_recue, date_reponse_entreprise, 
                    etape_atteinte, nom_coord_recruteur, notes_perso, resultat_final, 
                    nb_jours_candidature_reponse, nb_jours_candidature_resultat_final, 
                    score_adequation_poste_profil, priorite_offre, mots_cles_poste, 
                    motivation, 1 - (embedding <#> (:ref)::vector) AS simil_temp
                FROM {table_name}
            ) AS s
        ORDER BY simil_temp DESC
    """)
    
    with engine.connect() as conn:
        result = conn.execute(query, {"ref": ref_emb_str})       
        df_similarity = pd.DataFrame(result.fetchall(),columns=["id", "origine_annonce", "source", "recherche", "titre", 
                                                                "description", "entreprise", "lieu", "latitude", "longitude", 
                                                                "commune", "code_postal", "departement", "type_contrat_libelle", 
                                                                "date_publication", "url", "secteur_activites", "last_updated", 
                                                                "embedding", "similitude", "candidature_envisagee", "type_contrat", 
                                                                "experience_requise", "candidature_effectuee", "date_candidature", 
                                                                "nom_cv", "nom_lm", "nom_fichier_offre", "date_relance_prevue", 
                                                                "date_relance_effectuee", "reponse_recue", "date_reponse_entreprise", 
                                                                "etape_atteinte", "nom_coord_recruteur", "notes_perso", "resultat_final", 
                                                                "nb_jours_candidature_reponse", "nb_jours_candidature_resultat_final", 
                                                                "score_adequation_poste_profil", "priorite_offre", "mots_cles_poste", 
                                                                "motivation","simil_temp"])       
        df_similarity['similitude'] = df_similarity['simil_temp']
        df_similarity.drop(columns=['simil_temp'], inplace=True)
        update_similarity(df_similarity, engine, table_name)
        print("Le score de similarité a été mis à jour dans la BDD !")

### Sauvegarde CSV

In [1295]:
# # --- Sauvegarde Parquet ---
# def save_to_csv(df, csv_directory, filename):
#     if df.empty:
#         return
#     os.makedirs(csv_directory, exist_ok=True)
#     today = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
#     path_csv = os.path.join(csv_directory, f"{today}_{filename}.csv")
#     df.to_csv(path_csv, index=False,encoding="utf-8")
#     print(f"✅ Sauvegardé dans {path_csv}")

### Sauvegarde Parquet

In [1296]:
# # --- Sauvegarde Parquet ---
# def save_to_parquet(df,parquet_directory, filename):
#     if df.empty:
#         return
#     os.makedirs(parquet_directory, exist_ok=True)
#     today = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
#     path_parquet = os.path.join(parquet_directory, f"{today}_{filename}.parquet")
#     df.to_parquet(path_parquet, index=False)
#     print(f"✅ Sauvegardé dans {path_parquet}")

### Sauvegarde Excel

In [1297]:
# --- Sauvegarde Excel --- 
def save_to_excel(df,excel_directory, filename):
    if df.empty:
        return
    os.makedirs(excel_directory, exist_ok=True)
    today = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    path_excel = os.path.join(excel_directory, f"{today}_{filename}.xlsx")
    df.to_excel(path_excel, index=False)
    return path_excel

In [1298]:
def export_from_table_to_excel(engine, table_name,excel_directory, filename):
    query = text(f"""
        SELECT     
               id, origine_annonce, source, recherche, titre, 
               description, entreprise, lieu, latitude, longitude,
               commune, code_postal, departement,type_contrat_libelle,
               date_publication, url, secteur_activites,last_updated,
               embedding, similitude, candidature_envisagee, type_contrat, 
               experience_requise, candidature_effectuee, date_candidature, 
               nom_cv, nom_lm, nom_fichier_offre, date_relance_prevue, 
               date_relance_effectuee, reponse_recue, date_reponse_entreprise, 
               etape_atteinte, nom_coord_recruteur, notes_perso, resultat_final, 
               nb_jours_candidature_reponse, nb_jours_candidature_resultat_final, 
               score_adequation_poste_profil, priorite_offre, mots_cles_poste, 
               motivation
        FROM {table_name}
    """)
    
    with engine.connect() as conn:
        result = conn.execute(query)       
        df_export = pd.DataFrame(result.fetchall(),columns=["id", "origine_annonce", "source", "recherche", "titre", 
                                                                "description", "entreprise", "lieu", "latitude", "longitude", 
                                                                "commune", "code_postal", "departement", "type_contrat_libelle", 
                                                                "date_publication", "url", "secteur_activites", "last_updated", 
                                                                "embedding", "similitude", "candidature_envisagee", "type_contrat", 
                                                                "experience_requise", "candidature_effectuee", "date_candidature", 
                                                                "nom_cv", "nom_lm", "nom_fichier_offre", "date_relance_prevue", 
                                                                "date_relance_effectuee", "reponse_recue", "date_reponse_entreprise", 
                                                                "etape_atteinte", "nom_coord_recruteur", "notes_perso", "resultat_final", 
                                                                "nb_jours_candidature_reponse", "nb_jours_candidature_resultat_final", 
                                                                "score_adequation_poste_profil", "priorite_offre", "mots_cles_poste", 
                                                                "motivation"])       
        save_to_excel(df_export,excel_directory, filename)

### Mise à jour du fichier de suivi

In [1299]:
def update_tracking_file_scrapping(engine, tracking_file_path_dir,filename, table_name):   
    try:
        # Charger ton fichier de suivi existant
        os.makedirs(tracking_file_path_dir, exist_ok=True)
        tracking_file_path = os.path.join(tracking_file_path_dir, f"{filename}.xlsx")
        df = pd.read_excel(tracking_file_path)     

        if df.empty:
            logging.info("📭 DataFrame vide, rien à insérer.")
            return 0
        
        # Nettoyage des NaN
        df = df.where(pd.notnull(df), None)       
        
        metadata = MetaData()
        table = Table(table_name, metadata, autoload_with=engine)
        now = datetime.utcnow()
        count = 0
     
        with engine.begin() as conn:
            for row in df.to_dict(orient="records"):
                row["last_updated"] = now
                
                # # Calcul embedding uniquement si nouvelle offre
                # if not row.get("embedding"):
                #     row["embedding"] = compute_embedding(row["description"])
                    
                stmt = pg_insert(table).values(row)
                stmt = stmt.on_conflict_do_update(
                    index_elements=['id'],
                    set_={
                        'candidature_envisagee' : stmt.excluded.candidature_envisagee,
                        'type_contrat' : stmt.excluded.type_contrat,
                        'experience_requise' : stmt.excluded.experience_requise,
                        'candidature_effectuee' : stmt.excluded.candidature_effectuee,
                        'date_candidature' : stmt.excluded.date_candidature,
                        'nom_cv' : stmt.excluded.nom_cv,
                        'nom_lm' : stmt.excluded.nom_lm,
                        'nom_fichier_offre' : stmt.excluded.nom_fichier_offre,
                        'date_relance_prevue' : stmt.excluded.date_relance_prevue,
                        'date_relance_effectuee' : stmt.excluded.date_relance_effectuee,
                        'reponse_recue' : stmt.excluded.reponse_recue,
                        'date_reponse_entreprise' : stmt.excluded.date_reponse_entreprise,
                        'etape_atteinte' : stmt.excluded.etape_atteinte,
                        'nom_coord_recruteur' : stmt.excluded.nom_coord_recruteur,
                        'notes_perso' : stmt.excluded.notes_perso,
                        'resultat_final' : stmt.excluded.resultat_final,
                        'nb_jours_candidature_reponse' : stmt.excluded.nb_jours_candidature_reponse,
                        'nb_jours_candidature_resultat_final' : stmt.excluded.nb_jours_candidature_resultat_final,
                        'score_adequation_poste_profil' : stmt.excluded.score_adequation_poste_profil,
                        'priorite_offre' : stmt.excluded.priorite_offre,
                        'mots_cles_poste' : stmt.excluded.mots_cles_poste,
                        'motivation' : stmt.excluded.motivation,
                        'last_updated': now       
                    }
                )
                conn.execute(stmt)
                count += 1
        print(f"✅ BDD mise à jour à partir du fichier de suivi.")
        logging.info(f"{count} offres insérées/mises à jour dans PostgreSQL.")
        return count

    except SQLAlchemyError as e:
        logging.error(f"❌ Erreur lors de l'UPSERT : {str(e)}")
        return 0

### Pipeline principal

In [1300]:
# ---------------- PIPELINE ----------------
def run_pipeline_web_scrapping(query):
    logging.info("Début du pipeline WEB_SCRAPPING.")

    try:        
        print("Authentification France Travail...")
        token = get_ft_token()
    
        print("Récupération des offres France Travail...")
        ft_jobs = fetch_france_travail_jobs(query, token)
    
        print("Récupération des offres Adzuna...")
        adzuna_jobs = fetch_adzuna_jobs(query)
    
        print("Fusion et déduplication...")
        all_jobs = ft_jobs + adzuna_jobs
    
        if not all_jobs:
            print("⚠️ Aucune offre trouvée.")
            return
    
        print(f"Nombre d'offres d'emploi avant déduplication : {len(all_jobs)}")
        jobs_clean = deduplicate(all_jobs)
        print(f"Nombre d'offres d'emploi après déduplication : {len(jobs_clean)}")
    
        print("Affichage des offres...")
        df = pd.DataFrame(jobs_clean)

        print("Ajout commune, code_postal et departement...")
        df = get_localization_info(df)
    
        # Connexion DB
        print("Connexion à la base PostgreSQL...")
        engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
    
        # Initier la bdd
        api_table_name = 'web_scrapping_table'
        init_db(engine, api_table_name)
    
        print("💾 Sauvegarde en base PostgreSQL...")
        save_to_postgres_upsert_initial_api(df, engine, table_name=api_table_name)
        print("💾 Sauvegarde en base PostgreSQL TERMINEE !!!...")

        print("💾 Sauvegarde du score de similarité en base PostgreSQL...")
        compute_similarity(reference_text_clean, engine, api_table_name)
        print("💾 Sauvegarde du score de similarité en base PostgreSQL TERMINEE !!!...")

        print("FIN DU SCRIPT DE WEB SCRAPPING !!!...")
        
    except Exception as e:
        logging.exception(f"Pipeline échoué: {e}")

### Pipeline EXPORT

In [1301]:
# ---------------- PIPELINE ----------------
def run_pipeline_export_all(table_name):
    logging.info("Début du pipeline EXPORT.")

    try:           
        # Connexion DB
        print("Connexion à la base PostgreSQL...")
        engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
    
        # Initier la bdd
        init_db(engine, table_name)

        print("💾 Export de la BDD...")
        export_from_table_to_excel(engine, table_name,EXPORT_TRACKING_DIR_PROC, "export_base")
        print("💾 Export de la BDD TERMINé...")

        print("FIN DU SCRIPT D'EXPORTATION!!!...")
        
    except Exception as e:
        logging.exception(f"Pipeline échoué: {e}")

### Pipeline UPDATE

In [1302]:
# ---------------- PIPELINE ----------------
def run_pipeline_update_all(table_name):
    logging.info("Début du pipeline UPDATE.")

    try:           
        # Connexion DB
        print("Connexion à la base PostgreSQL...")
        engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

        # Initier la bdd
        init_db(engine, table_name)

        tracking_file_path_dir = IMPORT_TRACKING_DIR_PROC
        filename = "update_file"

        print("💾 Mise à jour de la BDD...")
        update_tracking_file_scrapping(engine, tracking_file_path_dir,filename,table_name)
        print("💾 Mise à jour de la BDD TERMINEE...")

        print("FIN DU SCRIPT D'UPDATE!!!...")
        
    except Exception as e:
        logging.exception(f"Pipeline échoué: {e}")

In [1303]:
# # Chargement depuis CSV
# today = datetime.now().strftime("%Y-%m-%d")
# path_parquet = os.path.join(PARQUET_DIR, f"{today}_offres.parquet")
# df = pd.read_parquet(path_parquet)
# display(df.shape)
# display(df.head(3))

### Procédure principale

In [1304]:
# ---------------------------
# MAIN
# ---------------------------
if __name__ == "__main__":
    JOB_QUERY = ["data analyst"]
    api_table_name = 'web_scrapping_table'
    
    for query in JOB_QUERY:
        # run_pipeline_web_scrapping(query)
        run_pipeline_update_all(api_table_name)
        run_pipeline_export_all(api_table_name)

Connexion à la base PostgreSQL...
💾 Mise à jour de la BDD...


/tmp/ipykernel_2101/1501279494.py:16: SAWarning: Did not recognize type 'vector' of column 'embedding'
  table = Table(table_name, metadata, autoload_with=engine)


✅ BDD mise à jour à partir du fichier de suivi.
💾 Mise à jour de la BDD TERMINEE...
FIN DU SCRIPT D'UPDATE!!!...
Connexion à la base PostgreSQL...
💾 Export de la BDD...
💾 Export de la BDD TERMINé...
FIN DU SCRIPT D'EXPORTATION!!!...
